In [1]:

import csv
from cqs.object import Instance
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.random import random_circuit
import qiskit.qasm3 as qasm3
from instances_b.reader_b import read_csv_b
from transpiler.transpile import transpile_circuit
from examples.benchmark.cqs_simulation import main_prober, main_solver

def __num_to_pauli_list(num_list):
    paulis = ['I', 'X', 'Y', 'Z']
    pauli_list = [paulis[int(i)] for i in num_list]
    return pauli_list

def __add_Pauli_gate(qc, which_qubit, which_gate):
    if which_gate == 0:
        qc.id(which_qubit)
    elif which_gate == 1:
        qc.x(which_qubit)
    elif which_gate == 2:
        qc.y(which_qubit)
    elif which_gate == 3:
        qc.z(which_qubit)
    else:
        return ValueError("Not supported Pauli gate type.")

def __num_to_pauli_circuit(num_list):
    n = len(num_list)
    num_list = [int(i) for i in num_list]
    qr = QuantumRegister(n, 'q')
    qc = QuantumCircuit(qr)
    for i in range(n):
        __add_Pauli_gate(qc, i, num_list[i])
    return qc

def create_random_circuit_in_native_gate(n, d):
    ub = random_circuit(num_qubits=n,max_operands=2, depth=d, measure=False)
    # ub = transpile_circuit(ub, device='Aria', optimization_level=2)
    return ub

In [2]:
with open('3_qubit_data_generation_matrix_A.csv', 'r', newline='') as csvfile:
    data_b = read_csv_b(3)
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for i, row in enumerate(reader):
        if 2 > i > 0:
            row_clean = [j for j in ''.join(row).split('"') if j != ',']
            nLc = row_clean[0].split(',')
            n = int(nLc[0])
            print("qubit number is:", n)
            L = int(nLc[1])
            print("term number is:", L)
            kappa = float(nLc[2])
            print('condition number is', kappa)
            pauli_strings = [__num_to_pauli_list(l) for l in eval(row_clean[1])]
            print('Pauli strings are:', pauli_strings)
            pauli_circuits = [__num_to_pauli_circuit(l) for l in eval(row_clean[1])]
            coeffs = [float(i) for i in eval(row_clean[2])]
            print('coefficients are:', coeffs)
            print()

            # circuit depth d
            d = 3
            ub = qasm3.loads(data_b.iloc[i].qasm)
            print('Ub is given by:', data_b.iloc[i].b)
            print(ub)

            # generate instance
            instance = Instance(n, L, kappa)
            instance.generate(given_coeffs=coeffs, given_unitaries=pauli_circuits, given_ub=ub)
            Itr, LOSS, ansatz_tree = main_prober(instance, backend='qiskit-noiseless',ITR=None,
                                    shots=0, optimization_level=2,
                                    noise_level_two_qubit=0, noise_level_one_qubit=0, readout_error=0)
            print(Itr)
            print(LOSS)
            print('ANsatz tree contains:')
            # for qc in ansatz_tree:
            #     print(qc)
ansatz_tree=[ansatz_tree[i] for i in range(len(ansatz_tree)-1)]

Resolved file path: C:\Users\Nikita\PycharmProjects\CQS_singapore\instances_b\3_b_random_circuits.csv
Reading file: C:\Users\Nikita\PycharmProjects\CQS_singapore\instances_b\3_b_random_circuits.csv
qubit number is: 3
term number is: 3
condition number is 3.5456037521362305
Pauli strings are: [['Y', 'Z', 'Y'], ['I', 'Y', 'Y'], ['X', 'Y', 'I']]
coefficients are: [-1.01, 1.17, 2.76]

Ub is given by: [(0.7071067811865475+0j), 0j, 0j, 0j, (-0.6099094210942649+0.357785547584665j), 0j, 0j, 0j]
                                           
q_0: ──────────X───────────■───────■───────
               │           │ ┌─────┴──────┐
q_1: ──────────X───────────■─┤ Rz(4.3897) ├
     ┌───────────────────┐   └────────────┘
q_2: ┤ U2(2.6111,5.8783) ├─────────────────
     └───────────────────┘                 


Hadamar tests Progress: 100%|██████████| 9/9 [00:01<00:00,  6.54it/s]
C:\Users\Nikita\PycharmProjects\CQS_singapore\cqs\optimization.py:91: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3729.)
  return torch.abs(0.5*torch.matmul(x.T, torch.matmul(Q, x)) + torch.matmul(r.T, x) + 1)


loss: 1.0
combination parameters are: [(8.875925078199192e-19+1.3466478242161492e-42j)]


Hadamar tests Progress: 100%|██████████| 27/27 [00:01<00:00, 15.27it/s]


loss: 0.23874246416144873
combination parameters are: [(2.020010211030527e-18+1.2240698113898509e-17j), (0.27581796050071716-3.0621938417859916e-17j)]


Hadamar tests Progress: 100%|██████████| 54/54 [00:03<00:00, 14.85it/s]


loss: 0.04036166897401017
combination parameters are: [(1.026312020034042e-17-0.09446865320205688j), (0.2758179008960724-4.2345036535259493e-17j), (5.660834344808286e-17-0.16955627501010895j)]


Hadamar tests Progress: 100%|██████████| 90/90 [00:06<00:00, 14.58it/s]


loss: 6.770118288201843e-08
combination parameters are: [(-1.8508507226844273e-10-0.09446866065263748j), (0.31842944025993347+2.9871154750844408e-15j), (-1.844130126382737e-10-0.16955626010894775j), (0.0764804258942604+4.868677618181938e-15j)]
[1, 2, 3, 4]
[1.0, 0.23874246416144873, 0.04036166897401017, 6.770118288201843e-08]
ANsatz tree contains:


In [5]:
loss, alphas = main_solver(instance, ansatz_tree, backend='qiskit-noisy', shots=1000, optimization_level=2,
                                       noise_level_two_qubit=0.0, noise_level_one_qubit=0.00, readout_error=0.1)
print(loss)
print(alphas)

Hadamar tests Progress: 100%|██████████| 135/135 [27:59<00:00, 12.44s/it]


loss: 0.15808322597437208
combination parameters are: [(0.0772351622581482-0.07723625004291534j), (0.3704848885536194+0.04854521155357361j), (0.2179139405488968-0.045877907425165176j), (0.1429031789302826+0.03197352588176727j), (-0.14953532814979553-0.14295275509357452j)]
0.15808322597437208
[(0.0772351622581482-0.07723625004291534j), (0.3704848885536194+0.04854521155357361j), (0.2179139405488968-0.045877907425165176j), (0.1429031789302826+0.03197352588176727j), (-0.14953532814979553-0.14295275509357452j)]
